In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 55.1 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.9 MB/s eta 0:00:00 0:00:01mm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 kB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 40.6 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install tf-keras

  Using cached tf_keras-2.18.0-py3-none-any.whl (1.7 MB)

[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 55.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 25.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 36.2 MB/s eta 0:00:00
  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
     ━━━━━━━━━━━━━━━━━━━━

In [13]:
!pip install transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 8.0 MB/s eta 0:00:0000:01

[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForQuestionAnswering
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer

import datetime


# Load the BERT model
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

# Load tokenizer to prepare SQUAD data into BERT undesrtandable input
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)

# Define a custom BERT model wrapper class like the ones in training.ipynb example
class BertQA(nn.Module):
    def __init__(self):
        super(BertQA, self).__init__()
        self.bert_qa = BertForQuestionAnswering.from_pretrained(model_name)
    
    def forward(self, input_ids, attention_mask, start_positions=None, end_positions=None):
        outputs = self.bert_qa(
            input_ids=input_ids,
            attention_mask=attention_mask,
            start_positions=start_positions,
            end_positions=end_positions
        )
        return outputs

# Initialize the model
model_name = "bert-base-uncased"
model = BertQA()

# Load SQUAD dataset
dataset = load_dataset("squad")

# Function for data preprocessing: Turn SQUAD [context,question,answer] into BERT tokens
def preprocess_data(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = [c.strip() for c in examples["context"]]

    # Tokenize data for BERT
    tokenized_examples = tokenizer(
        questions,
        contexts,
        truncation="only_second",          # If context is too long, truncate it
        max_length=384,                    # Maximum length of token
        stride=128,                        # Allows to reatin context of previous tokenization
        return_overflowing_tokens=False,   # False for working with smaller datasets, True for better performance and working with bigger sets
        return_offsets_mapping=True,       # Enable character index offsets
        padding="max_length"               # Add padding for uniform input size
    )
    
    # Map answer positions to tokenized positions
    offset_mapping = tokenized_examples.pop("offset_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        if len(examples["answers"]) > i:                    # If there is an answer POSITION for the example
            answer = examples["answers"][i].get("text", []) # Get the answer text
            
            if answer:  # If there is TEXT for the answer, find the tokens that correspond to the answer
                answer = answer[0]
                start_char = examples["answers"][i].get("answer_start", [0])[0]  # Get the answer's first character offset in the original context
                end_char = start_char + len(answer)                              # Get the answer's last character offset 
                start_token = None
                end_token = None

                # Iterate the tokens to find which ones correspong to the answer
                for j, (start, end) in enumerate(offsets):
                    if start_char >= start and start_char < end:
                        start_token = j
                    if end_char > start and end_char <= end:
                        end_token = j
                        break
                
                if start_token is not None and end_token is not None:   # If the answer tokens were found, add them
                    tokenized_examples["start_positions"].append(start_token)
                    tokenized_examples["end_positions"].append(end_token)
                else:                                                   # If the answer tokens were NOT found, add a default value of 0
                    tokenized_examples["start_positions"].append(0)
                    tokenized_examples["end_positions"].append(0)
            else:     # If there is no TEXT for answer, add a default value of 0
                tokenized_examples["start_positions"].append(0)
                tokenized_examples["end_positions"].append(0)
        else: # If there is no POSITION for the example's answer, add a default value of 0
            tokenized_examples["start_positions"].append(0)
            tokenized_examples["end_positions"].append(0)

    return tokenized_examples


# Preprocess the train set
train_dataset = dataset["train"].map(preprocess_data, batched=True, remove_columns=dataset["train"].column_names) # Working with big datasets
# train_dataset = dataset["train"].shuffle(seed=42).select(range(1000)).map(preprocess_data, batched=True, remove_columns=dataset["train"].column_names) # Working with smaller datasets

# Set up DataLoader, handling variable sequence lengths
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator) # Batch size >8 gives me CUDA out of memory :$

# Training parameters
epochs = 1
learning_rate = 3e-5
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate) # Try Adam, SDG, and lr schedulers

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Start:",datetime.datetime.now())

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    i = 0
    running_loss = 0
    for batch in train_loader:
        running_loss = 0
        # Move batch to device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        start_positions = batch["start_positions"].to(device)
        end_positions = batch["end_positions"].to(device)
        
        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Print execution progress every 100 steps. 
        
        # Track the loss
        total_loss += loss.item()
        running_loss += loss.item()
        i += 1
        if i % 100 == 99:    # Imprime la pérdida cada 100 mini-batches
            print('[%d, %5d] running_loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
    
    # Print the average loss for this epoch
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss}")
print("End:",datetime.datetime.now())


2024-12-16 21:35:59.052235: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-16 21:35:59.065403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734381359.081991 1956026 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734381359.087022 1956026 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-16 21:35:59.104189: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Start: 2024-12-16 21:36:07.880877
[1,   100] running_loss: 0.042
[1,   200] running_loss: 0.031
[1,   300] running_loss: 0.024
[1,   400] running_loss: 0.023
[1,   500] running_loss: 0.019
[1,   600] running_loss: 0.026
[1,   700] running_loss: 0.018
[1,   800] running_loss: 0.022
[1,   900] running_loss: 0.020
[1,  1000] running_loss: 0.017
[1,  1100] running_loss: 0.019
[1,  1200] running_loss: 0.026
[1,  1300] running_loss: 0.017
[1,  1400] running_loss: 0.019
[1,  1500] running_loss: 0.017
[1,  1600] running_loss: 0.008
[1,  1700] running_loss: 0.019
[1,  1800] running_loss: 0.017
[1,  1900] running_loss: 0.028
[1,  2000] running_loss: 0.013
[1,  2100] running_loss: 0.015
[1,  2200] running_loss: 0.018
[1,  2300] running_loss: 0.024
[1,  2400] running_loss: 0.013
[1,  2500] running_loss: 0.027
[1,  2600] running_loss: 0.007
[1,  2700] running_loss: 0.018
[1,  2800] running_loss: 0.017
[1,  2900] running_loss: 0.015
[1,  3000] running_loss: 0.024
[1,  3100] running_loss: 0.015
[1,  